In [4]:
import os
import subprocess
import pandas as pd
import time
import seaborn as sns
sns.set(style='whitegrid')
sns.set(rc={'figure.figsize':(11.7, 8.27)})

def run_experiment(exp_name: str, rps=100, duration_sec=5, handler='sleep50', silent=True) -> str:
    dump_file = f'/tmp/{exp_name}.bin'
    run_vegeta = f"echo 'GET http://localhost:8890/{handler}' | vegeta -cpus 1 attack -rate {rps} -duration {duration_sec}s -timeout 1s -name {exp_name} -workers 1 > {dump_file}"
    p = subprocess.run(run_vegeta, shell=True)
    if not silent:
        print(p)
        print('---'*10)
        print_report = f"cat {dump_file} | vegeta report -reporter text > /tmp/res.txt"
        subprocess.run(print_report, shell=True)
        print(open('/tmp/res.txt').read())

def read_experiment(csv_path: str, warmup_time_sec = 0.2) -> pd.DataFrame:
    names = ['unix_ts_ns', 'http_code', 'latency_ns', 'bytes_out', 'bytes_in', 'x', 'error', 'exp_name', 'y']
    data = pd.read_csv(csv_path, header=None, names=names)
    del data['error']
    del data['x']
    del data['y']
    del data['bytes_out']
    del data['bytes_in']
    begin_ts_ns = data.unix_ts_ns.min()
    exp_start_ts_ns = begin_ts_ns + int(warmup_time_sec * 1_000_000_000)
    data = data[data.unix_ts_ns > exp_start_ts_ns]
    data.reset_index(drop=True, inplace=True)
    data['latency_ms'] = (data['latency_ns'] / 1_000_000).round(2)
    del data['unix_ts_ns']
    del data['latency_ns']
    return data

def aggr_exp(exp_names: list) -> str:
    out_csv = f'/tmp/out.csv'
    dump_files = ','.join([f'/tmp/{exp_name}.bin' for exp_name in exp_names])
    dump_ext = f"vegeta dump -inputs {dump_files} -output {out_csv} -dumper csv"
    p = subprocess.run(dump_ext, shell=True)
    # print(p)
    return out_csv

def plot_exp_data(exp_data):
    ax = sns.boxplot(x='latency_ms', y='exp_name', data=exp_data)

def latency_table(exp_data, rps):
    a = exp_data.groupby('exp_name').quantile([.95, .98, .99])
    a = a.reset_index().pivot(index='exp_name', columns='level_1', values='latency_ms')
    a = a.round(1)
    df = a.reset_index()
    #df['exp_name'], df['rps'] = df.exp_name.str.split('_rps_').str
    df['rps'] = rps
    df['rps'] = df['rps'].astype(int)
    df.sort_values('rps', inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df[['rps','exp_name', 0.95, 0.98, 0.99]]
    return df


def hand_experiment(name: str, rps=100, duration_sec=3, repeats=5, **kwargs):
    exp_datas = []
    latencies = []
    for _ in range(repeats):
        run_experiment(name, rps, duration_sec, **kwargs)
        csv_file = aggr_exp([name])
        exp_data = read_experiment(csv_file, warmup_time_sec=1)
        exp_datas.append(exp_data)
        df = latency_table(exp_data, rps)
        latencies.append(df)
    latencies = pd.concat(latencies)
    return latencies, exp_datas


In [5]:
lats = []
repeats = 3
duration_sec = 10
rps = 1000

# Tornado

In [6]:
lat, _ = hand_experiment('tornado_oldstyle', rps=rps, duration_sec=duration_sec, repeats=repeats, handler='oldstyle50')
lats.append(lat)
time.sleep(0.5)
lat

level_1,rps,exp_name,0.95,0.98,0.99
0,1000,tornado_oldstyle,51.8,52.0,53.1
0,1000,tornado_oldstyle,54.3,55.4,56.1
0,1000,tornado_oldstyle,51.9,52.1,52.6
0,1000,tornado_async,51.8,52.9,54.4
0,1000,tornado_async,52.4,53.1,53.6
0,1000,tornado_async,51.8,52.6,54.0


In [ ]:
lat, _ = hand_experiment('tornado_async', rps=rps, duration_sec=duration_sec, repeats=repeats)
time.sleep(0.5)
lats.append(lat)
lat

In [15]:
rps = 1000
lat, _ = hand_experiment('tornado_hard_work', rps=rps, duration_sec=duration_sec, repeats=repeats, handler='hard_work')
lats.append(lat)
time.sleep(0.5)
lat

level_1,rps,exp_name,0.95,0.98,0.99
0,1000,tornado_hard_work,132.6,134.4,135.6
0,1000,tornado_hard_work,133.9,135.4,136.8
0,1000,tornado_hard_work,130.9,132.1,133.0


In [58]:
rps = 1200
lat, _ = hand_experiment('tornado_hard_work', rps=rps, duration_sec=duration_sec, repeats=repeats, handler='hard_work')
lats.append(lat)
time.sleep(0.5)
lat
# too many open files

level_1,rps,exp_name,0.95,0.98,0.99
0,1200,tornado_hard_work,173.1,177.1,180.3
0,1200,tornado_hard_work,166.8,172.1,174.6
0,1200,tornado_hard_work,186.5,190.6,193.2


In [17]:
lat, _ = hand_experiment('tornado_hard_work_oldstyle', rps=rps, duration_sec=duration_sec, repeats=repeats, handler='hard_work_oldstyle')
lats.append(lat)
time.sleep(0.5)
lat

level_1,rps,exp_name,0.95,0.98,0.99
0,1000,tornado_hard_work_oldstyle,179.3,185.1,188.9
0,1000,tornado_hard_work_oldstyle,176.3,182.1,185.1
0,1000,tornado_hard_work_oldstyle,182.2,189.3,192.9


# aiohttp

In [7]:
lat, _ = hand_experiment('aiohttp', rps=rps, duration_sec=duration_sec, repeats=repeats)
lats.append(lat)
time.sleep(0.5)

In [12]:
lat, _ = hand_experiment('aiohttp_hard_work', rps=rps, duration_sec=duration_sec, repeats=repeats, handler='hard_work')
lats.append(lat)
lat

level_1,rps,exp_name,0.95,0.98,0.99
0,1000,aiohttp_hard_work,128.5,128.6,128.7
0,1000,aiohttp_hard_work,128.5,128.6,129.2
0,1000,aiohttp_hard_work,128.5,128.6,129.5


In [19]:
rps = 1500
lat, _ = hand_experiment('aiohttp_hard_work', rps=rps, duration_sec=duration_sec, repeats=repeats, handler='hard_work')
lats.append(lat)
lat

level_1,rps,exp_name,0.95,0.98,0.99
0,1500,aiohttp_hard_work,128.1,129.7,131.2
0,1500,aiohttp_hard_work,128.3,130.5,132.3
0,1500,aiohttp_hard_work,128.1,129.8,131.2


In [21]:
rps = 2000
lat, _ = hand_experiment('aiohttp_hard_work', rps=rps, duration_sec=duration_sec, repeats=repeats, handler='hard_work')
lats.append(lat)
lat

level_1,rps,exp_name,0.95,0.98,0.99
0,2000,aiohttp_hard_work,131.6,134.8,137.4
0,2000,aiohttp_hard_work,131.6,134.7,136.9
0,2000,aiohttp_hard_work,133.0,136.9,140.3


In [62]:
rps = 2500
lat, _ = hand_experiment('aiohttp_hard_work', rps=rps, duration_sec=duration_sec, repeats=repeats, handler='hard_work')
lats.append(lat)
lat

level_1,rps,exp_name,0.95,0.98,0.99
0,2500,aiohttp_hard_work,173.8,179.8,184.9
0,2500,aiohttp_hard_work,147.8,151.4,153.3
0,2500,aiohttp_hard_work,161.3,167.2,173.8


In [63]:
rps = 3000
lat, _ = hand_experiment('aiohttp_hard_work', rps=rps, duration_sec=duration_sec, repeats=repeats, handler='hard_work')
lats.append(lat)
lat

level_1,rps,exp_name,0.95,0.98,0.99
0,3000,aiohttp_hard_work,268.4,280.2,287.8
0,3000,aiohttp_hard_work,260.9,274.4,281.8
0,3000,aiohttp_hard_work,216.8,223.3,227.9


In [68]:
def results(lats: list) -> pd.DataFrame:
    df = pd.concat(lats)
    df = df[df[0.99] < 200]
    df = df.groupby(['exp_name', 'rps']).mean().sort_values(0.98).round(1)
    df = df.reset_index()
    return df

In [69]:
res = results(lats)
res.style.bar(subset=[0.98, 0.99], color='#d65f5f')

level_1,exp_name,rps,0.95,0.98,0.99
0,aiohttp,1000,51.5,51.6,51.6
1,tornado_async,1000,52,52.9,54
2,tornado_oldstyle,1000,52.7,53.2,53.9
3,aiohttp_hard_work,1000,128.5,128.6,129.1
4,aiohttp_hard_work,1500,128.2,130,131.6
5,tornado_hard_work,1000,132.5,134,135.1
6,aiohttp_hard_work,2000,132.1,135.5,138.2
7,aiohttp_hard_work,2500,161,166.1,170.7
8,tornado_hard_work,1200,175.5,179.9,182.7
9,tornado_hard_work_oldstyle,1000,179.3,185.5,189


In [65]:
res = results(lats)
res = res[res.exp_name.str.contains('hard_work')]
res.style.bar(subset=[0.98, 0.99], color='#d65f5f')

level_1,exp_name,rps,0.95,0.98,0.99
3,aiohttp_hard_work,1000,128.5,128.6,129.1
4,aiohttp_hard_work,1500,128.2,130,131.6
5,tornado_hard_work,1000,132.5,134,135.1
6,aiohttp_hard_work,2000,132.1,135.5,138.2
7,aiohttp_hard_work,2500,161,166.1,170.7
8,tornado_hard_work,1200,175.5,179.9,182.7
9,tornado_hard_work_oldstyle,1000,179.3,185.5,189


In [11]:
# mean cpu time of process